In [10]:
from pydantic import BaseModel
from typing import List, Optional
from langchain_core.messages import BaseMessage

class AgentState(BaseModel):
    messages: List[BaseMessage]
    tool_calls: list = []
    final_answer: Optional[str] = None


In [11]:
import json
from fastapi.responses import StreamingResponse
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langchain.tools import tool

from src.core.utils.logger import get_logger

logger = get_logger(__name__)

@tool
def get_weather(location: str) -> str:
    """Get the weather for a location"""
    # Fake weather data
    return f"The weather in {location} is sunny and 25°C"

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def search_web(query: str) -> str:
    """Search the web for information"""
    return f"Search results for '{query}': Found 10 results about {query}"

tools = [get_weather, calculate, search_web]



In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.7
).bind_tools(tools)


In [13]:
async def llm_node(state: AgentState) -> AgentState:
    response = await llm.ainvoke(state.messages)

    state.messages.append(response)

    if response.tool_calls:
        state.tool_calls = response.tool_calls

    else:
        state.final_answer = response.content

    return state


In [14]:
from langchain_core.messages import ToolMessage

async def tool_node(state: AgentState) -> AgentState:
    for call in state.tool_calls:
        tool_name = call["name"]
        tool_args = call.get("args", {})

        for tool in tools:
            if tool.name == tool_name:
                result = tool.invoke(tool_args)

                state.messages.append(
                    ToolMessage(
                        content=str(result),
                        tool_call_id=call["id"]
                    )
                )

    state.tool_calls = []
    return state


In [15]:
def route(state: AgentState) -> str:
    if state.tool_calls:
        return "tool"
    if state.final_answer:
        return "end"
    return "llm"


In [16]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("llm", llm_node)
graph.add_node("tool", tool_node)

graph.set_entry_point("llm")

graph.add_conditional_edges(
    "llm",
    route,
    {
        "tool": "tool",
        "end": END
    }
)

graph.add_edge("tool", "llm")

app = graph.compile()


In [18]:
from langchain_core.messages import HumanMessage

state = AgentState(
    messages=[HumanMessage(content="What's the weather in Hanoi?")]
)

final_state = await app.ainvoke(state)

print(final_state)


{'messages': [HumanMessage(content="What's the weather in Hanoi?", additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 88, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-CmBETjhSu5hcx0kgezn5UOsntsj6M', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b15e6-0ae7-79c3-92ac-735f3b9947ee-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Hanoi'}, 'id': 'call_NO67Y1coQEBA5d0V0kQRWJao', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 15, 'total_tokens': 103, 'input_token_details': {'aud

In [20]:
for message in final_state['messages']:
    print(message.content)

What's the weather in Hanoi?

The weather in Hanoi is sunny and 25°C
The weather in Hanoi is sunny with a temperature of 25°C.


In [ ]:
final_state.